In [1]:
!git clone https://github.com/allenai/bilm-tf.git

Cloning into 'bilm-tf'...
remote: Total 292 (delta 0), reused 0 (delta 0), pack-reused 292
Receiving objects: 100% (292/292), 588.40 KiB | 645.00 KiB/s, done.
Resolving deltas: 100% (137/137), done.


In [2]:
cd /content/bilm-tf

/content/bilm-tf


In [3]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating bilm.egg-info
writing bilm.egg-info/PKG-INFO
writing dependency_links to bilm.egg-info/dependency_links.txt
writing requirements to bilm.egg-info/requires.txt
writing top-level names to bilm.egg-info/top_level.txt
writing manifest file 'bilm.egg-info/SOURCES.txt'
writing manifest file 'bilm.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/bilm
copying bilm/__init__.py -> build/lib/bilm
copying bilm/training.py -> build/lib/bilm
copying bilm/data.py -> build/lib/bilm
copying bilm/elmo.py -> build/lib/bilm
copying bilm/model.py -> build/lib/bilm
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/bilm
copying build/lib/bilm/__init__.py -> build/bdist.linux-x86_64/egg/bilm
copying build/lib/bilm/training.py -> build/bdist.linux-x86_64/egg/bilm
copying buil

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#json_file = '{"lstm": {"use_skip_connections": true, "projection_dim": 500, "cell_clip": 3, "proj_clip": 3, "dim": 1024, "n_layers": 2}, "char_cnn": {"activation": "relu", "filters": [[1, 32], [2, 32], [3, 64], [4, 128], [5, 256], [6, 512], [7, 1024]], "n_highway": 1, "embedding": {"dim": 16}, "n_characters": 262, "max_characters_per_token": 15}}'

#with open("/content/drive/My Drive/elmo/checkpoint/options.json", "w") as fp:
#    fp.write(json_file)

In [0]:
vocab_file = '/content/drive/My Drive/FYP/elmo/roman_urdu/vocab.txt'
options_file = '/content/drive/My Drive/FYP/elmo/roman_urdu/options.json'
weight_file = '/content/drive/My Drive/FYP/elmo/roman_urdu/weights.hdf5'
outfile = '/content/drive/My Drive/FYP/elmo/roman_urdu/embeddings.hdf5'

In [0]:
!cp -f "/content/drive/My Drive/elmo/roman_urdu/embeddings.hdf5" "/content/outfile.hdf5"

In [9]:
import tensorflow as tf
tf.get_variable_scope().reuse_variables()

from bilm import TokenBatcher, BidirectionalLanguageModel, weight_layers, dump_token_embeddings

In [0]:
# Create a TokenBatcher to map text to token ids.
batcher = TokenBatcher(vocab_file)

# Input placeholders to the biLM.
tf.reset_default_graph()
context_token_ids = tf.placeholder('int32', shape=(None, None))

# Build the biLM graph.
bilm = BidirectionalLanguageModel(
    options_file,
    weight_file,
    use_character_inputs=False,
    embedding_weight_file=outfile
)

In [23]:
context_embeddings_op = bilm(context_token_ids)

USING SKIP CONNECTIONS


## Older code below

In [0]:
#raw_context = ['yeh aam si baat hai', 'mujhay aam khana pasand hai']
raw_context = ["is mausam mai aam khanay ka alag hi maza hai", "yeh us ke liyay aam si baat hai"]
tokenized_context = [sentence.split() for sentence in raw_context]

In [25]:
# Get an op to compute ELMo (weighted average of the internal biLM layers)

elmo_context_input = weight_layers('input', context_embeddings_op, l2_coef=0.0)

with tf.Session() as sess:
    # It is necessary to initialize variables once before running inference.
    sess.run(tf.global_variables_initializer())
 
    # Create batches of data.
    context_ids = batcher.batch_sentences(tokenized_context)
    print("Shape of context ids = ", context_ids.shape)
 
    # Compute ELMo representations (here for the input only, for simplicity).
    elmo_context_input_ = sess.run(
        elmo_context_input['weighted_op'],
        feed_dict={context_token_ids: context_ids}
    )
 
print("Shape of generated embeddings = ",elmo_context_input_.shape)

Shape of context ids =  (2, 12)
Shape of generated embeddings =  (2, 10, 1000)


In [0]:
#There are 2 sentences with max sentence length of 10 and embedding length for each word
#is 1000 (500 dim concatenated forward and backward). Hence, the shape of generated embeddings are (2, 5, 1000).

In [17]:
print (elmo_context_input_[1,1,:])

[ 0.24410954 -0.1224492  -0.02628822 -0.10620669  0.200198   -0.07283232
 -0.08321205  0.4265794   0.18039575  0.13399181  0.27462363 -0.28415778
  0.22968692  0.18139543 -0.03349497  0.02482393 -0.0491108   0.12439024
 -0.3043028   0.40329498 -0.16232607 -0.19930404  0.16962183  0.1369648
 -0.52164835 -0.37357697 -0.19598675 -0.30066046 -0.5583079  -0.16792682
 -0.05908423 -0.04375581 -0.00777739 -0.14088961  0.11879101 -0.5144002
  0.00434463 -0.24722879 -0.3346116   0.1415599   0.14261399 -0.06574731
  0.09280755 -0.12100472  0.14669245  0.30131328  0.23899508  0.38805774
 -0.32387185  0.14983326  0.1905972  -0.24968514 -0.20737043  0.21469113
 -0.05573745  0.1377769   0.17284788  0.0442032   0.06396861  0.27766907
 -0.3205003   0.73601854 -0.02138119 -0.2473445   0.14661542  0.17180811
 -0.45953596  0.25644353 -0.22625506 -0.06599385 -0.07694655 -0.07362304
 -0.21898681  0.16544269  0.06367126 -0.11733048  0.19508763 -0.41288823
  0.17507221 -0.14567748 -0.14671335 -0.17820378  0.3

In [18]:
import numpy as np
import scipy.spatial.distance as ds

# Computing euclidean distance between words embedding
euc_dist_bet_aam = np.linalg.norm(elmo_context_input_[0,1,:]-elmo_context_input_[1,1,:])
#euc_dist_bet_computer_fashion = np.linalg.norm(elmo_context_input_[1,5,:]-elmo_context_input_[2,0,:])

# Computing cosine distance between words embedding
cos_dist_bet_aam = ds.cosine(elmo_context_input_[0,1,:],elmo_context_input_[1,1,:])
#cos_dist_bet_computer_fashion = ds.cosine(elmo_context_input_[1,5,:],elmo_context_input_[2,0,:])
 
print("Euclidean Distance Comparison - ")
print("\naam(baat) - aam(khana) = ",np.round(euc_dist_bet_aam,2))
print("\n\nCosine Distance Comparison - ")
print("\naam(baat) - aam(khana) = ",np.round(cos_dist_bet_aam,2))

Euclidean Distance Comparison - 

aam(baat) - aam(khana) =  5.13


Cosine Distance Comparison - 

aam(baat) - aam(khana) =  0.18


In [26]:
from numpy.linalg import norm

a = np.array(elmo_context_input_[0,3,:])
b = np.array(elmo_context_input_[1,4,:])
l1_a = norm(a, 1)
l1_b = norm(b, 1)
l2_a = norm(a)
l2_b = norm(b)

print ("l1_a: ", l1_a)
print ("l1_b: ", l1_b)
print ("l2_a: ", l2_a)
print ("l2_b: ", l2_b)


l1_a:  200.4241
l1_b:  221.77567
l2_a:  7.980165
l2_b:  8.77106
